# Scoring Review: Retrieval Augmented Generation (RAG)

In this notebook, we will demonstrate how reviewers can use RAG to perform their reviews by referencing appropriate external context per item.

## Setting up the notebook

High-level configs

In [1]:
%reload_ext autoreload
%autoreload 2

from dotenv import load_dotenv

# Load environment variables from .env file. Adjust the path to the .env file as needed.
load_dotenv(dotenv_path='../.env')

# Enable asyncio in Jupyter
import asyncio
import nest_asyncio

nest_asyncio.apply()

#  Add the package to the path (required if you are running this notebook from the examples folder)
import sys
sys.path.append('../../')


Import required packages

In [8]:
import json
import numpy as np
from openai import AsyncOpenAI
import pandas as pd
from pydantic import BaseModel
from tqdm.auto import tqdm

from lattereview.providers import OpenAIProvider
from lattereview.providers import LiteLLMProvider
from lattereview.agents import ScoringReviewer
from lattereview.workflows import ReviewWorkflow

## Data

Building five example stories and dummy question-answering pairs from each story:

In [3]:
class BuildStoryOutput(BaseModel):
    story: str
    questions: list[str]
    answers: list[bool]

async def build_story():
    prompt = """
    Write a one-paragraph story with whatever realistic or imaginary theme you like,  
    then create three TRUE/FALSE questions based on your story. 
    Ensure that only readers of your story can determine whether the statements are true or false. 
    Do not reveal the answers to your questions.
    Return your story, a Python list of three questions, and another Python list of three boolean responses to the questions as your output.
    """
    provider = OpenAIProvider(model="gpt-4o", response_format_class=BuildStoryOutput)
    return await provider.get_json_response(prompt, temperature=0.9)

def run_build_story():
    response =  asyncio.run(build_story())[0]
    return response

data = {
    "question": [],
    "answer": [],
    "story": []
}
for i in tqdm(range(5)):
    out = json.loads(run_build_story())
    for j in range(3):
        data["question"].append(out["questions"][j])
        data["answer"].append(out["answers"][j])
        data["story"].append(out["story"])


data = pd.DataFrame(data)
data.to_csv("data.csv", index=False)
data

  0%|          | 0/5 [00:00<?, ?it/s]

,question,answer,story
0,The secret garden was located on the rooftop o...,True,"In the bustling city of Verdantopolis, where t..."
1,Elio allowed the mayor to take the Night Bloom...,False,"In the bustling city of Verdantopolis, where t..."
2,The Night Blooming Fern blooms under the light...,True,"In the bustling city of Verdantopolis, where t..."
3,The Whispering Elm's leaves were golden in color.,False,"In the heart of an ancient forest, there stood..."
4,The tree murmured at dusk.,True,"In the heart of an ancient forest, there stood..."
5,Elara planned to visit the tree again.,True,"In the heart of an ancient forest, there stood..."
6,Lyra's companion is named Nutmeg.,True,"In the heart of the verdant Elvenwood Forest, ..."
7,The Well of Whimsy is said to grant the power ...,False,"In the heart of the verdant Elvenwood Forest, ..."
8,The crow claimed to have seen dragons nesting ...,True,"In the heart of the verdant Elvenwood Forest, ..."
9,Was the waterfall in the Enchanted Forest said...,True,"In the heart of the Enchanted Forest, there la..."


Embedding the stories to build a vector base:

In [4]:
data = pd.read_csv("data.csv")

async def get_embedding(text):
    client = AsyncOpenAI()
    if isinstance(text, str):
        text = [text]
    text = [x.replace("\n", " ") for x in text]         
    out = await client.embeddings.create(
        model="text-embedding-ada-002",
        input=text,
        encoding_format="float"
    )
    out = [np.array(x.embedding) for x in out.data]
    return out if len(out) > 1 else out[0]

stories = {story: None for story in set(data["story"].tolist())}

# Create async tasks for all embeddings
async def process_embeddings():
    tasks = [get_embedding(story) for story in stories.keys()]
    embeddings = await asyncio.gather(*tasks)
    return list(zip(embeddings, stories.keys()))

# Run the async code and get results
vector_story_pairs = await process_embeddings()
vector_base = np.array([x[0] for x in vector_story_pairs])
vector_base

array([[ 0.02013032, -0.00834996, -0.01089617, ...,  0.00205425,
         0.00239829, -0.03922355],
       [ 0.01107596, -0.00785628, -0.00699814, ..., -0.01633787,
        -0.00084982, -0.01649752],
       [-0.00901955,  0.00349605, -0.01779216, ...,  0.00381121,
        -0.00939644, -0.03594822],
       [ 0.01213862,  0.00485675, -0.02092641, ...,  0.00364419,
         0.00725579, -0.03768058],
       [ 0.00509998, -0.02309775, -0.01362209, ..., -0.01088438,
        -0.00843241, -0.02378882]])

## Retrieval
Here, we will define a simple retrieval function that takes an input text (in our case, a statement) as input, embeds it, and finds the reference text (in our case, a story) with the highest cosine similarity to the embedding of that statement. The output of this function can then be used by the reviewer agent as additional context for each item it reviews.

In [5]:
async def find_relevant_story(statement):
    s_embeddings = await get_embedding(statement)
    dot_product = np.dot(vector_base, s_embeddings)
    base_norms = np.linalg.norm(vector_base, axis=1)
    query_norm = np.linalg.norm(s_embeddings)
    cosine_similarities = dot_product / (base_norms * query_norm)
    retrieved_index = np.argmax(cosine_similarities)
    retrieved_story = vector_story_pairs[retrieved_index][1]
    return retrieved_story

input_index = 11
statement = data.iloc[input_index]["question"]
retrieved_story = await find_relevant_story(statement)

print(f"=== The question was chosen from row {input_index} ===\n{statement}")
print(f"=== The related story to the question ===\n{data.iloc[input_index]['story']}")
print(f"=== The retrieved Story ===\n{retrieved_story}")

=== The question was chosen from row 11 ===
Did the creature emerge from the waterfall to grant Aria's wish?
=== The related story to the question ===
In the heart of the Enchanted Forest, there lay a hidden waterfall that sparkled with the colors of the rainbow. Legend had it that this waterfall was home to a mystical creature named Lumi, a creature of light and wonder who could grant wishes. One sunny day, a young girl named Aria ventured deep into the forest with her loyal dog, Toby, in search of the waterfall. After hours of walking, they finally reached the magical place. Aria whispered her wish into the mist, hoping that Lumi would hear her plea. To her delight, a glimmering figure emerged from the waterfall, its presence filling the air with a gentle glow. Aria’s heart skipped with joy as she realized her wish might come true.
=== The retrieved Story ===
In the heart of the Enchanted Forest, there lay a hidden waterfall that sparkled with the colors of the rainbow. Legend had it

## Scoring with Retrieval Augmented Generation

Here, we will create a simple reviewer agent that processes each input statement. The key argument we define for this reviewer is the `additional_context` argument. Note how we pass the `find_relevant_story` function to this `additional_context` argument, enabling the reviewer to apply this function to each input item. This allows the agent to retrieve the most relevant context for each statement before conducting its review. In our case, the reviewer will decide whether the statement is true or false based on the retrieved context, which, in this case, is the corresponding story.

In [6]:
reviewer = ScoringReviewer(
    provider=LiteLLMProvider(model="gpt-4o-mini"),
    name="reviewer",
    max_concurrent_requests=20, 
    backstory="A frequent book reader",
    input_description="TRUE/FALSE questions about stories",
    model_args={"max_tokens": 200, "temperature": 0.1},
    reasoning = "brief",
    scoring_task="Decide if the input statement is True or False given the provided story in the provided context",
    scoring_set=[1, 2],
    scoring_rules='Score 1 if the statement is TRUE and 2 if the statement is FALSE.',
    additional_context = find_relevant_story
)

review = ReviewWorkflow(
    workflow_schema=[
        {
            "round": 'A',
            "reviewers": [reviewer],
            "text_inputs": ["question"]
        }
    ]
)

updated_data = asyncio.run(review(data))
updated_data


====== Starting review round A (1/1) ======

Processing 15 eligible rows


['round: A', 'reviewer_name: reviewer'] -                     2025-01-04 21:13:13: 100%|██████████| 15/15 [00:02<00:00,  5.38it/s]

The following columns are present in the dataframe at the end of reviewer's reivew in round A: ['question', 'answer', 'story', 'round-A_reviewer_output', 'round-A_reviewer_reasoning', 'round-A_reviewer_score', 'round-A_reviewer_certainty']


,question,answer,story,round-A_reviewer_output,round-A_reviewer_reasoning,round-A_reviewer_score,round-A_reviewer_certainty
0,The secret garden was located on the rooftop o...,True,"In the bustling city of Verdantopolis, where t...",{'reasoning': 'The statement incorrectly claim...,The statement incorrectly claims that the secr...,1,90
1,Elio allowed the mayor to take the Night Bloom...,False,"In the bustling city of Verdantopolis, where t...",{'reasoning': 'The statement is FALSE because ...,The statement is FALSE because Elio refused th...,2,90
2,The Night Blooming Fern blooms under the light...,True,"In the bustling city of Verdantopolis, where t...",{'reasoning': 'The statement is TRUE because t...,The statement is TRUE because the context clea...,1,95
3,The Whispering Elm's leaves were golden in color.,False,"In the heart of an ancient forest, there stood...",{'reasoning': 'The statement claims that the W...,The statement claims that the Whispering Elm's...,2,95
4,The tree murmured at dusk.,True,"In the heart of an ancient forest, there stood...",{'reasoning': 'The statement 'The tree murmure...,The statement 'The tree murmured at dusk' is t...,1,90
5,Elara planned to visit the tree again.,True,"In the heart of an ancient forest, there stood...",{'reasoning': 'The statement is TRUE because E...,The statement is TRUE because Elara vowed to r...,1,90
6,Lyra's companion is named Nutmeg.,True,"In the heart of the verdant Elvenwood Forest, ...",{'reasoning': 'The statement is FALSE because ...,The statement is FALSE because Lyra's companio...,2,90
7,The Well of Whimsy is said to grant the power ...,False,"In the heart of the verdant Elvenwood Forest, ...",{'reasoning': 'The statement about the Well of...,The statement about the Well of Whimsy grantin...,2,90
8,The crow claimed to have seen dragons nesting ...,True,"In the heart of the verdant Elvenwood Forest, ...",{'reasoning': 'The statement is TRUE because t...,The statement is TRUE because the talking crow...,1,90
9,Was the waterfall in the Enchanted Forest said...,True,"In the heart of the Enchanted Forest, there la...",{'reasoning': 'The statement is TRUE because t...,The statement is TRUE because the provided con...,1,100


Last but not least, you can check the memory of the agent for every item it has reviewed.

In [7]:
# An example review item (pass the index for a row in the input dataframe)

reviewer.memory[11]

{'system_prompt': "Your name is: <<reviewer>> Your backstory is: <<A frequent book reader>>. Your task is to review input itmes with the following description: <<TRUE/FALSE questions about stories>>. Your final output should have the following keys: reasoning (<class 'str'>), score (<class 'int'>), certainty (<class 'int'>).",
 'model_args': {'max_tokens': 200, 'temperature': 0.1},
 'input_prompt': "**Review the input item below and complete the scoring task as instructed:** --- **Input item:** <<Review Task ID: A-11 === question === Did the creature emerge from the waterfall to grant Aria's wish?>> **Scoring task:** <<Decide if the input statement is True or False given the provided story in the provided context>> --- **Instructions:** 1. **Score** the input item using only the values in this set: [1, 2]. 2. Follow these rules when determining your score: <<Score 1 if the statement is TRUE and 2 if the statement is FALSE.>>. 3. After assigning a score, report your certainty level as a